# An Even Easier Introduction to CUDA

This notebook accompanies Mark Harris's popular blog post [_An Even Easier Introduction to CUDA_](https://developer.nvidia.com/blog/even-easier-introduction-cuda/).

If you enjoy this notebook and want to learn more, the [NVIDIA DLI](https://nvidia.com/dli) offers several in depth CUDA Programming courses.

For those of you just starting out, please consider [_Fundamentals of Accelerated Computing with CUDA C/C++_](https://courses.nvidia.com/courses/course-v1:DLI+C-AC-01+V1/about) which provides dedicated GPU resources, a more sophisticated programming environment, use of the [NVIDIA Nsight Systems™](https://developer.nvidia.com/nsight-systems) visual profiler, dozens of interactive exercises, detailed presentations, over 8 hours of material, and the ability to earn a DLI Certificate of Competency.

Similarly, for Python programmers, please consider [_Fundamentals of Accelerated Computing with CUDA Python_](https://courses.nvidia.com/courses/course-v1:DLI+C-AC-02+V1/about).

For more intermediate and advance CUDA programming materials, please check out the _Accelerated Computing_ section of the NVIDIA DLI [self-paced catalog](https://www.nvidia.com/en-us/training/online/).

<img src="https://developer.download.nvidia.com/training/courses/T-AC-01-V1/CUDA_Cube_1K.jpeg" width="400">

This post is a super simple introduction to CUDA, the popular parallel computing platform and programming model from NVIDIA. I wrote a previous [“Easy Introduction”](https://developer.nvidia.com/blog/easy-introduction-cuda-c-and-c/) to CUDA in 2013 that has been very popular over the years. But CUDA programming has gotten easier, and GPUs have gotten much faster, so it’s time for an updated (and even easier) introduction.

CUDA C++ is just one of the ways you can create massively parallel applications with CUDA. It lets you use the powerful C++ programming language to develop high performance algorithms accelerated by thousands of parallel threads running on GPUs. Many developers have accelerated their computation- and bandwidth-hungry applications this way, including the libraries and frameworks that underpin the ongoing revolution in artificial intelligence known as [Deep Learning](https://developer.nvidia.com/deep-learning).

So, you’ve heard about CUDA and you are interested in learning how to use it in your own applications. If you are a C or C++ programmer, this blog post should give you a good start. To follow along, you’ll need a computer with an CUDA-capable GPU (Windows, Mac, or Linux, and any NVIDIA GPU should do), or a cloud instance with GPUs (AWS, Azure, IBM SoftLayer, and other cloud service providers have them). You’ll also need the free [CUDA Toolkit](https://developer.nvidia.com/cuda-toolkit) installed.

Let's get started!

<img src="https://developer-blogs.nvidia.com/wp-content/uploads/2017/01/cuda_ai_cube-625x625.jpg" width="400">

## Starting Simple

We’ll start with a simple C++ program that adds the elements of two arrays with a million elements each.

In [1]:
%%writefile add.cpp

#include <iostream>
#include <math.h>

// function to add the elements of two arrays
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20; // 1M elements

  float *x = new float[N];
  float *y = new float[N];

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the CPU
  add(N, x, y);

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  delete [] x;
  delete [] y;

  return 0;
}

Overwriting add.cpp


Executing the above cell will save its contents to the file add.cpp.

The following cell will compile and run this C++ program.

In [2]:
%%shell
g++ add.cpp -o add

Then run it:

In [3]:
%%shell
./add

Max error: 0


As expected, it prints that there was no error in the summation and then exits. Now I want to get this computation running (in parallel) on the many cores of a GPU. It’s actually pretty easy to take the first steps.

First, I just have to turn our `add` function into a function that the GPU can run, called a *kernel* in CUDA. To do this, all I have to do is add the specifier `__global__` to the function, which tells the CUDA C++ compiler that this is a function that runs on the GPU and can be called from CPU code.

```cpp
// CUDA Kernel function to add the elements of two arrays on the GPU
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}
```

These `__global__` functions are known as *kernels*, and code that runs on the GPU is often called *device code*, while code that runs on the CPU is *host code*.

## Memory Allocation in CUDA

To compute on the GPU, I need to allocate memory accessible by the GPU. [Unified Memory](https://developer.nvidia.com/blog/unified-memory-in-cuda-6/) in CUDA makes this easy by providing a single memory space accessible by all GPUs and CPUs in your system. To allocate data in unified memory, call `cudaMallocManaged()`, which returns a pointer that you can access from host (CPU) code or device (GPU) code. To free the data, just pass the pointer to `cudaFree()`.

I just need to replace the calls to `new` in the code above with calls to `cudaMallocManaged()`, and replace calls to `delete []` with calls to `cudaFree`.

```cpp
  // Allocate Unified Memory -- accessible from CPU or GPU
  float *x, *y;
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  ...

  // Free memory
  cudaFree(x);
  cudaFree(y);
```

Finally, I need to *launch* the `add()` kernel, which invokes it on the GPU. CUDA kernel launches are specified using the triple angle bracket syntax `<<< >>>`. I just have to add it to the call to `add` before the parameter list.

```cpp
add<<<1, 1>>>(N, x, y);
```

Easy! I’ll get into the details of what goes inside the angle brackets soon; for now all you need to know is that this line launches one GPU thread to run `add()`.

Just one more thing: I need the CPU to wait until the kernel is done before it accesses the results (because CUDA kernel launches don’t block the calling CPU thread). To do this I just call `cudaDeviceSynchronize()` before doing the final error checking on the CPU.

Here’s the complete code:

In [4]:
%%writefile add.cu

#include <iostream>
#include <math.h>

#define CHECK_CUDA_ERROR(call) \
do { \
  cudaError_t err = call; \
  if (err != cudaSuccess) { \
    std::cerr << "CUDA error in " << __FILE__ << ":" << __LINE__ << ": " \
              << cudaGetErrorString(err) << std::endl; \
    exit(1); \
  } \
} while(0)

// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  CHECK_CUDA_ERROR(cudaMallocManaged(&x, N*sizeof(float)));
  CHECK_CUDA_ERROR(cudaMallocManaged(&y, N*sizeof(float)));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, x, y);
  CHECK_CUDA_ERROR(cudaGetLastError());

  // Wait for GPU to finish before accessing on host
  CHECK_CUDA_ERROR(cudaDeviceSynchronize());

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Overwriting add.cu


### Check GPU and CUDA Version Compatibility

In [5]:
%%shell
# Check NVIDIA GPU and driver info
nvidia-smi

# Check NVCC version
nvcc --version

Sat Dec  6 04:10:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
%%shell
# Compile with architecture flags compatible with Colab GPUs
# Use compute capability 7.5 for T4 or 8.0 for A100
nvcc -arch=sm_75 add.cu -o add_cuda
./add_cuda

Max error: 0


This is only a first step, because as written, this kernel is only correct for a single thread, since every thread that runs it will perform the add on the whole array. Moreover, there is a [race condition](https://en.wikipedia.org/wiki/Race_condition) since multiple parallel threads would both read and write the same locations.

## Profile it!

I think the simplest way to find out how long the kernel takes to run is to run it with `nvprof`, the command line GPU profiler that comes with the CUDA Toolkit. Just type `nvprof ./add_cuda` on the command line:

In [7]:
%%shell

nvprof ./add_cuda

==5574== NVPROF is profiling process 5574, command: ./add_cuda
Max error: 0
==5574== Profiling application: ./add_cuda
==5574== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  108.92ms         1  108.92ms  108.92ms  108.92ms  add(int, float*, float*)
      API calls:   52.76%  108.93ms         1  108.93ms  108.93ms  108.93ms  cudaDeviceSynchronize
                   46.85%  96.731ms         2  48.365ms  42.156us  96.688ms  cudaMallocManaged
                    0.23%  481.93us         2  240.96us  225.86us  256.07us  cudaFree
                    0.08%  161.19us         1  161.19us  161.19us  161.19us  cudaLaunchKernel
                    0.07%  151.75us       114  1.3310us     109ns  66.339us  cuDeviceGetAttribute
                    0.01%  14.157us         1  14.157us  14.157us  14.157us  cuDeviceGetName
                    0.00%  5.7520us         1  5.7520us  5.7520us  5.7520us  cuDeviceGetPCIBusId
        

The above will show the single call to `add`. Your timing may vary depending on the GPU allocated to you by Colab. To see the current GPU allocated to you run the following cell and look in the `Name` column where you might see, for example `Tesla T4`:

In [8]:
%%shell

nvidia-smi

Sat Dec  6 04:10:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P0             25W /   70W |       0MiB /  15360MiB |     58%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Let's make it faster with parallelism.

## Picking up the Threads

Now that you’ve run a kernel with one thread that does some computation, how do you make it parallel? The key is in CUDA’s `<<<1, 1>>>` syntax. This is called the execution configuration, and it tells the CUDA runtime how many parallel threads to use for the launch on the GPU. There are two parameters here, but let’s start by changing the second one: the number of threads in a thread block. CUDA GPUs run kernels using blocks of threads that are a multiple of 32 in size, so 256 threads is a reasonable size to choose.

```cpp
add<<<1, 256>>>(N, x, y);
```

If I run the code with only this change, it will do the computation once per thread, rather than spreading the computation across the parallel threads. To do it properly, I need to modify the kernel. CUDA C++ provides keywords that let kernels get the indices of the running threads. Specifically, `threadIdx.x` contains the index of the current thread within its block, and `blockDim.x` contains the number of threads in the block. I’ll just modify the loop to stride through the array with parallel threads.

```cpp
__global__
void add(int n, float *x, float *y)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      y[i] = x[i] + y[i];
}
```

The `add` function hasn’t changed that much. In fact, setting `index` to 0 and `stride` to 1 makes it semantically identical to the first version.

Here we save the file as add_block.cu and compile and run it in `nvprof` again.

In [9]:
%%writefile add_block.cu

#include <iostream>
#include <math.h>

#define CHECK_CUDA_ERROR(call) \
do { \
  cudaError_t err = call; \
  if (err != cudaSuccess) { \
    std::cerr << "CUDA error in " << __FILE__ << ":" << __LINE__ << ": " \
              << cudaGetErrorString(err) << std::endl; \
    exit(1); \
  } \
} while(0)

__global__
void add(int n, float *x, float *y)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  CHECK_CUDA_ERROR(cudaMallocManaged(&x, N*sizeof(float)));
  CHECK_CUDA_ERROR(cudaMallocManaged(&y, N*sizeof(float)));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 256>>>(N, x, y);
  CHECK_CUDA_ERROR(cudaGetLastError());

  // Wait for GPU to finish before accessing on host
  CHECK_CUDA_ERROR(cudaDeviceSynchronize());

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Overwriting add_block.cu


In [10]:
%%shell

nvcc -arch=sm_75 add_block.cu -o add_block
nvprof ./add_block

==5631== NVPROF is profiling process 5631, command: ./add_block
Max error: 0
==5631== Profiling application: ./add_block
==5631== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.5265ms         1  3.5265ms  3.5265ms  3.5265ms  add(int, float*, float*)
      API calls:   96.02%  104.75ms         2  52.374ms  60.512us  104.69ms  cudaMallocManaged
                    3.24%  3.5330ms         1  3.5330ms  3.5330ms  3.5330ms  cudaDeviceSynchronize
                    0.43%  472.73us         2  236.37us  225.91us  246.82us  cudaFree
                    0.16%  174.51us         1  174.51us  174.51us  174.51us  cudaLaunchKernel
                    0.13%  137.10us       114  1.2020us     105ns  56.892us  cuDeviceGetAttribute
                    0.01%  12.162us         1  12.162us  12.162us  12.162us  cuDeviceGetName
                    0.00%  4.6430us         1  4.6430us  4.6430us  4.6430us  cuDeviceGetPCIBusId
      

That’s a big speedup (compare the time for the `add` kernel by looking at the `GPU activities` field), but not surprising since I went from 1 thread to 256 threads. Let’s keep going to get even more performance.

## Out of the Blocks

CUDA GPUs have many parallel processors grouped into Streaming Multiprocessors, or SMs. Each SM can run multiple concurrent thread blocks. As an example, a Tesla P100 GPU based on the [Pascal GPU Architecture](https://developer.nvidia.com/blog/inside-pascal/) has 56 SMs, each capable of supporting up to 2048 active threads. To take full advantage of all these threads, I should launch the kernel with multiple thread blocks.

By now you may have guessed that the first parameter of the execution configuration specifies the number of thread blocks. Together, the blocks of parallel threads make up what is known as the *grid*. Since I have `N` elements to process, and 256 threads per block, I just need to calculate the number of blocks to get at least `N` threads. I simply divide `N` by the block size (being careful to round up in case `N` is not a multiple of `blockSize`).

```cpp
int blockSize = 256;
int numBlocks = (N + blockSize - 1) / blockSize;
add<<<numBlocks, blockSize>>>(N, x, y);
```

<img src="https://developer-blogs.nvidia.com/wp-content/uploads/2017/01/cuda_indexing.png" width="800">

I also need to update the kernel code to take into account the entire grid of thread blocks. CUDA provides `gridDim.x`, which contains the number of blocks in the grid, and `blockIdx.x`, which contains the index of the current thread block in the grid. Figure 1 illustrates the the approach to indexing into an array (one-dimensional) in CUDA using `blockDim.x`, `gridDim.x`, and `threadIdx.x`. The idea is that each thread gets its index by computing the offset to the beginning of its block (the block index times the block size: `blockIdx.x * blockDim.x`) and adding the thread’s index within the block (`threadIdx.x`). The code `blockIdx.x * blockDim.x + threadIdx.x` is idiomatic CUDA.

```cpp
__global__
void add(int n, float *x, float *y)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}
```

The updated kernel also sets stride to the total number of threads in the grid (`blockDim.x * gridDim.x`). This type of loop in a CUDA kernel is often called a [*grid-stride*](https://developer.nvidia.com/blog/cuda-pro-tip-write-flexible-kernels-grid-stride-loops/) loop.

Save the file as `add_grid.cu` and compile and run it in `nvprof` again.

In [11]:
%%writefile add_grid.cu

#include <iostream>
#include <math.h>

#define CHECK_CUDA_ERROR(call) \
do { \
  cudaError_t err = call; \
  if (err != cudaSuccess) { \
    std::cerr << "CUDA error in " << __FILE__ << ":" << __LINE__ << ": " \
              << cudaGetErrorString(err) << std::endl; \
    exit(1); \
  } \
} while(0)

__global__
void add(int n, float *x, float *y)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  CHECK_CUDA_ERROR(cudaMallocManaged(&x, N*sizeof(float)));
  CHECK_CUDA_ERROR(cudaMallocManaged(&y, N*sizeof(float)));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  int blockSize = 256;
  int numBlocks = (N + blockSize - 1) / blockSize;
  add<<<numBlocks, blockSize>>>(N, x, y);
  CHECK_CUDA_ERROR(cudaGetLastError());

  // Wait for GPU to finish before accessing on host
  CHECK_CUDA_ERROR(cudaDeviceSynchronize());

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Overwriting add_grid.cu


In [12]:
%%shell

nvcc -arch=sm_75 add_grid.cu -o add_grid
nvprof ./add_grid

==5683== NVPROF is profiling process 5683, command: ./add_grid
Max error: 0
==5683== Profiling application: ./add_grid
==5683== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  2.3388ms         1  2.3388ms  2.3388ms  2.3388ms  add(int, float*, float*)
      API calls:   96.91%  101.08ms         2  50.540ms  68.583us  101.01ms  cudaMallocManaged
                    2.26%  2.3596ms         1  2.3596ms  2.3596ms  2.3596ms  cudaDeviceSynchronize
                    0.51%  529.82us         2  264.91us  227.95us  301.87us  cudaFree
                    0.17%  176.12us         1  176.12us  176.12us  176.12us  cudaLaunchKernel
                    0.13%  136.40us       114  1.1960us     103ns  55.690us  cuDeviceGetAttribute
                    0.01%  13.271us         1  13.271us  13.271us  13.271us  cuDeviceGetName
                    0.01%  5.2830us         1  5.2830us  5.2830us  5.2830us  cuDeviceGetPCIBusId
        

That's another big speedup from running multiple blocks! (Note your results may vary from the blog post due to whatever GPU you've been allocated by Colab. If you notice your speedups for the final example are not as drastic as those in the blog post, check out #4 in the *Exercises* section below.)

## Exercises

To keep you going, here are a few things to try on your own.

1. Browse the [CUDA Toolkit documentation](https://docs.nvidia.com/cuda/index.html). If you haven’t installed CUDA yet, check out the [Quick Start Guide](https://docs.nvidia.com/cuda/cuda-quick-start-guide/index.html) and the installation guides. Then browse the [Programming Guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html) and the [Best Practices Guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html). There are also tuning guides for various architectures.
2. Experiment with `printf()` inside the kernel. Try printing out the values of `threadIdx.x` and `blockIdx.x` for some or all of the threads. Do they print in sequential order? Why or why not?
3. Print the value of `threadIdx.y` or `threadIdx.z` (or `blockIdx.y`) in the kernel. (Likewise for `blockDim` and `gridDim`). Why do these exist? How do you get them to take on values other than 0 (1 for the dims)?
4. If you have access to a [Pascal-based GPU](https://developer.nvidia.com/blog/inside-pascal/), try running `add_grid.cu` on it. Is performance better or worse than the K80 results? Why? (Hint: read about [Pascal’s Page Migration Engine and the CUDA 8 Unified Memory API](https://developer.nvidia.com/blog/beyond-gpu-memory-limits-unified-memory-pascal/).) For a detailed answer to this question, see the post [Unified Memory for CUDA Beginners](https://developer.nvidia.com/blog/unified-memory-cuda-beginners/).

## Exercise Solutions

Let's work through the exercises to deepen our understanding of CUDA programming.

### Exercise 2: Experiment with `printf()` inside the kernel

Let's create a version of our kernel that prints thread and block indices to understand the execution order.

In [13]:
%%writefile printf_exercise.cu

#include <iostream>
#include <math.h>

#define CHECK_CUDA_ERROR(call) \
do { \
  cudaError_t err = call; \
  if (err != cudaSuccess) { \
    std::cerr << "CUDA error in " << __FILE__ << ":" << __LINE__ << ": " \
              << cudaGetErrorString(err) << std::endl; \
    exit(1); \
  } \
} while(0)

// Kernel function with printf to show thread execution order
__global__
void add_with_printf(int n, float *x, float *y)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

  // Print only for the first few threads to avoid overwhelming output
  if (index < 10) {
    printf("Thread %d: blockIdx.x=%d, threadIdx.x=%d, computing index=%d\n",
           index, blockIdx.x, threadIdx.x, index);
  }

  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory
  CHECK_CUDA_ERROR(cudaMallocManaged(&x, N*sizeof(float)));
  CHECK_CUDA_ERROR(cudaMallocManaged(&y, N*sizeof(float)));

  // Initialize arrays
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel with multiple blocks
  int blockSize = 256;
  int numBlocks = (N + blockSize - 1) / blockSize;
  add_with_printf<<<numBlocks, blockSize>>>(N, x, y);
  CHECK_CUDA_ERROR(cudaGetLastError());

  // Wait for GPU to finish
  CHECK_CUDA_ERROR(cudaDeviceSynchronize());

  // Check for errors
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Overwriting printf_exercise.cu


In [14]:
%%shell

nvcc -arch=sm_75 printf_exercise.cu -o printf_exercise
./printf_exercise

Thread 0: blockIdx.x=0, threadIdx.x=0, computing index=0
Thread 1: blockIdx.x=0, threadIdx.x=1, computing index=1
Thread 2: blockIdx.x=0, threadIdx.x=2, computing index=2
Thread 3: blockIdx.x=0, threadIdx.x=3, computing index=3
Thread 4: blockIdx.x=0, threadIdx.x=4, computing index=4
Thread 5: blockIdx.x=0, threadIdx.x=5, computing index=5
Thread 6: blockIdx.x=0, threadIdx.x=6, computing index=6
Thread 7: blockIdx.x=0, threadIdx.x=7, computing index=7
Thread 8: blockIdx.x=0, threadIdx.x=8, computing index=8
Thread 9: blockIdx.x=0, threadIdx.x=9, computing index=9
Max error: 0


**Observation:** The threads do NOT print in sequential order! This is because:

1. **Parallel Execution**: Threads execute in parallel on different cores/SMs
2. **No Synchronization**: There's no guarantee about the order of execution between different blocks or even threads within a block
3. **Warp Scheduling**: The GPU scheduler organizes threads into warps (groups of 32) and executes them when resources are available
4. **Hardware Dependent**: The order can vary based on SM availability, workload, and GPU architecture

This demonstrates that CUDA threads execute asynchronously and independently.

### Exercise 3: Exploring Multi-Dimensional Thread Indexing

CUDA supports 3D grids and 3D blocks. Let's explore `threadIdx.y`, `threadIdx.z`, `blockIdx.y`, `blockDim.y`, and `gridDim.y`.

In [15]:
%%writefile multidim_exercise.cu

#include <iostream>
#include <math.h>

#define CHECK_CUDA_ERROR(call) \
do { \
  cudaError_t err = call; \
  if (err != cudaSuccess) { \
    std::cerr << "CUDA error in " << __FILE__ << ":" << __LINE__ << ": " \
              << cudaGetErrorString(err) << std::endl; \
    exit(1); \
  } \
} while(0)

// 2D kernel using multi-dimensional grids and blocks
__global__
void add_2d(int n, float *x, float *y)
{
  // Calculate 1D index from 2D grid structure
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  int idy = blockIdx.y * blockDim.y + threadIdx.y;

  // Convert 2D coordinates to 1D array index
  int gridWidth = gridDim.x * blockDim.x;
  int index = idy * gridWidth + idx;

  // Grid-stride loop to handle arrays larger than grid
  int stride_x = blockDim.x * gridDim.x;
  int stride_y = blockDim.y * gridDim.y;

  // Print info for a few threads
  if (idx < 2 && idy < 2) {
    printf("Thread (%d,%d): blockIdx=(%d,%d), threadIdx=(%d,%d), index=%d\n",
           idx, idy, blockIdx.x, blockIdx.y, threadIdx.x, threadIdx.y, index);
  }

  // Process elements with 2D stride
  for (int j = idy; j < n; j += stride_y) {
    for (int i = idx; i < n; i += stride_x) {
      int pos = j * n + i;
      if (pos < n * n)
        y[pos] = x[pos] + y[pos];
    }
  }
}

int main(void)
{
  int N = 1<<10; // Use smaller N for 2D (1024x1024 elements)
  float *x, *y;

  // Allocate Unified Memory
  CHECK_CUDA_ERROR(cudaMallocManaged(&x, N*N*sizeof(float)));
  CHECK_CUDA_ERROR(cudaMallocManaged(&y, N*N*sizeof(float)));

  // Initialize arrays
  for (int i = 0; i < N*N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Launch with 2D grid and 2D blocks
  dim3 threadsPerBlock(16, 16);
  dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                 (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

  std::cout << "Launching kernel with grid(" << numBlocks.x << "," << numBlocks.y
            << ") and blocks(" << threadsPerBlock.x << "," << threadsPerBlock.y << ")\n";

  add_2d<<<numBlocks, threadsPerBlock>>>(N, x, y);
  CHECK_CUDA_ERROR(cudaGetLastError());

  // Wait for GPU to finish
  CHECK_CUDA_ERROR(cudaDeviceSynchronize());

  // Check for errors
  float maxError = 0.0f;
  for (int i = 0; i < N*N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Overwriting multidim_exercise.cu


In [16]:
%%shell

nvcc -arch=sm_75 multidim_exercise.cu -o multidim_exercise
./multidim_exercise

Launching kernel with grid(64,64) and blocks(16,16)
Thread (0,0): blockIdx=(0,0), threadIdx=(0,0), index=0
Thread (1,0): blockIdx=(0,0), threadIdx=(1,0), index=1
Thread (0,1): blockIdx=(0,0), threadIdx=(0,1), index=1024
Thread (1,1): blockIdx=(0,0), threadIdx=(1,1), index=1025
Max error: 0


**Why do multi-dimensional indices exist?**

1. **Natural Mapping**: Many problems are naturally 2D (images, matrices) or 3D (volumes, simulations)
2. **Code Clarity**: Using 2D/3D indexing makes code more readable for matrix/image operations
3. **Memory Access Patterns**: Helps optimize memory coalescing by matching problem structure

**How to use them:**

- Use `dim3` type to specify grid and block dimensions:
  ```cpp
  dim3 blockSize(16, 16, 1);    // 16x16x1 = 256 threads per block
  dim3 gridSize(32, 32, 1);     // 32x32x1 = 1024 blocks
  kernel<<<gridSize, blockSize>>>(...)
  ```

**Default values:** If not specified, dimensions default to 1. In our 1D examples:
- `threadIdx.y = 0`, `threadIdx.z = 0`
- `blockDim.y = 1`, `blockDim.z = 1`
- Similar for `blockIdx` and `gridDim`

### Exercise 4: GPU Architecture and Performance Analysis

Let's check what GPU is available and analyze the performance characteristics.

In [17]:
%%shell

# Check GPU information
nvidia-smi --query-gpu=name,compute_cap,memory.total --format=csv

name, compute_cap, memory.total [MiB]
Tesla T4, 7.5, 15360 MiB


In [18]:
%%writefile gpu_info.cu

#include <iostream>
#include <cuda_runtime.h>

int main() {
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);

    for (int dev = 0; dev < deviceCount; dev++) {
        cudaDeviceProp prop;
        cudaGetDeviceProperties(&prop, dev);

        std::cout << "\n=== GPU Device " << dev << ": " << prop.name << " ===\n";
        std::cout << "Compute Capability: " << prop.major << "." << prop.minor << "\n";
        std::cout << "Total Global Memory: " << prop.totalGlobalMem / (1024*1024) << " MB\n";
        std::cout << "Multiprocessors (SMs): " << prop.multiProcessorCount << "\n";
        std::cout << "Max Threads per Block: " << prop.maxThreadsPerBlock << "\n";
        std::cout << "Max Threads per SM: " << prop.maxThreadsPerMultiProcessor << "\n";
        std::cout << "Warp Size: " << prop.warpSize << "\n";
        std::cout << "Memory Clock Rate: " << prop.memoryClockRate / 1000 << " MHz\n";
        std::cout << "Memory Bus Width: " << prop.memoryBusWidth << " bits\n";
        std::cout << "L2 Cache Size: " << prop.l2CacheSize / 1024 << " KB\n";

        // Determine architecture
        std::cout << "\nArchitecture: ";
        if (prop.major == 6 && prop.minor == 0) std::cout << "Pascal (P100)\n";
        else if (prop.major == 6 && prop.minor == 1) std::cout << "Pascal (GP10x)\n";
        else if (prop.major == 7 && prop.minor == 0) std::cout << "Volta (V100)\n";
        else if (prop.major == 7 && prop.minor == 5) std::cout << "Turing (T4/RTX 20xx)\n";
        else if (prop.major == 8 && prop.minor == 0) std::cout << "Ampere (A100)\n";
        else if (prop.major == 8 && prop.minor == 6) std::cout << "Ampere (RTX 30xx)\n";
        else if (prop.major == 8 && prop.minor == 9) std::cout << "Ada Lovelace (RTX 40xx)\n";
        else if (prop.major == 9 && prop.minor == 0) std::cout << "Hopper (H100)\n";
        else std::cout << "Unknown\n";

        // Check for Unified Memory features
        std::cout << "\nUnified Memory Features:\n";
        std::cout << "Managed Memory: " << (prop.managedMemory ? "Yes" : "No") << "\n";
        std::cout << "Concurrent Managed Access: " << (prop.concurrentManagedAccess ? "Yes" : "No") << "\n";
        std::cout << "Page Migration (Pascal+): " << (prop.major >= 6 ? "Yes" : "No") << "\n";
    }

    return 0;
}

Overwriting gpu_info.cu


In [19]:
%%shell

nvcc -arch=sm_75 gpu_info.cu -o gpu_info
./gpu_info


=== GPU Device 0: Tesla T4 ===
Compute Capability: 7.5
Total Global Memory: 15095 MB
Multiprocessors (SMs): 40
Max Threads per Block: 1024
Max Threads per SM: 1024
Warp Size: 32
Memory Clock Rate: 5001 MHz
Memory Bus Width: 256 bits
L2 Cache Size: 4096 KB

Architecture: Turing (T4/RTX 20xx)

Unified Memory Features:
Managed Memory: Yes
Concurrent Managed Access: Yes
Page Migration (Pascal+): Yes


**Performance Comparison: Pascal vs K80 (and newer architectures)**

The question asks about Pascal GPUs vs K80. Here's why Pascal and newer GPUs perform better with Unified Memory:

**K80 (Kepler, Compute 3.7):**
- Unified Memory works but with limitations
- No automatic page migration
- All managed memory pages must fit in GPU memory OR be explicitly managed
- CPU access causes GPU to wait (blocking)

**Pascal (Compute 6.x) and newer:**
- **Page Migration Engine**: Automatically migrates memory pages between CPU and GPU
- **On-demand paging**: Pages transferred only when needed
- **Oversubscription**: Total managed memory can exceed GPU memory
- **Concurrent access**: CPU and GPU can access different pages simultaneously
- **Better performance**: Reduced transfer overhead and smarter migration

**Why Pascal+ is faster:**
1. Hardware page fault handling
2. Automatic data prefetching
3. Better memory coherency
4. Reduced explicit synchronization needs

For GPUs with Compute Capability 6.0+, Unified Memory is significantly more efficient!

### Bonus: Performance Comparison with Different Memory Management

Let's compare Unified Memory vs explicit memory management to see the performance difference.

In [20]:
%%writefile compare_memory.cu

#include <iostream>
#include <math.h>
#include <chrono>

#define CHECK_CUDA_ERROR(call) \
do { \
  cudaError_t err = call; \
  if (err != cudaSuccess) { \
    std::cerr << "CUDA error in " << __FILE__ << ":" << __LINE__ << ": " \
              << cudaGetErrorString(err) << std::endl; \
    exit(1); \
  } \
} while(0)

__global__
void add(int n, float *x, float *y)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}

void test_unified_memory(int N, int blockSize, int numBlocks) {
  float *x, *y;

  auto start = std::chrono::high_resolution_clock::now();

  // Allocate Unified Memory
  CHECK_CUDA_ERROR(cudaMallocManaged(&x, N*sizeof(float)));
  CHECK_CUDA_ERROR(cudaMallocManaged(&y, N*sizeof(float)));

  // Initialize
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel
  add<<<numBlocks, blockSize>>>(N, x, y);
  CHECK_CUDA_ERROR(cudaGetLastError());
  CHECK_CUDA_ERROR(cudaDeviceSynchronize());

  auto end = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double> diff = end - start;

  // Verify
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));

  std::cout << "Unified Memory - Time: " << diff.count() << "s, Max error: " << maxError << std::endl;

  cudaFree(x);
  cudaFree(y);
}

void test_explicit_memory(int N, int blockSize, int numBlocks) {
  float *x, *y;        // Host pointers
  float *d_x, *d_y;    // Device pointers

  auto start = std::chrono::high_resolution_clock::now();

  // Allocate host memory
  x = (float*)malloc(N*sizeof(float));
  y = (float*)malloc(N*sizeof(float));

  // Initialize on host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Allocate device memory
  CHECK_CUDA_ERROR(cudaMalloc(&d_x, N*sizeof(float)));
  CHECK_CUDA_ERROR(cudaMalloc(&d_y, N*sizeof(float)));

  // Copy to device
  CHECK_CUDA_ERROR(cudaMemcpy(d_x, x, N*sizeof(float), cudaMemcpyHostToDevice));
  CHECK_CUDA_ERROR(cudaMemcpy(d_y, y, N*sizeof(float), cudaMemcpyHostToDevice));

  // Run kernel
  add<<<numBlocks, blockSize>>>(N, d_x, d_y);
  CHECK_CUDA_ERROR(cudaGetLastError());

  // Copy back to host
  CHECK_CUDA_ERROR(cudaMemcpy(y, d_y, N*sizeof(float), cudaMemcpyDeviceToHost));
  CHECK_CUDA_ERROR(cudaDeviceSynchronize());

  auto end = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double> diff = end - start;

  // Verify
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));

  std::cout << "Explicit Memory - Time: " << diff.count() << "s, Max error: " << maxError << std::endl;

  cudaFree(d_x);
  cudaFree(d_y);
  free(x);
  free(y);
}

int main(void)
{
  int N = 1<<20;
  int blockSize = 256;
  int numBlocks = (N + blockSize - 1) / blockSize;

  std::cout << "Comparing Unified Memory vs Explicit Memory Management\n";
  std::cout << "Array size: " << N << " elements\n\n";

  test_unified_memory(N, blockSize, numBlocks);
  test_explicit_memory(N, blockSize, numBlocks);

  return 0;
}

Overwriting compare_memory.cu


In [21]:
%%shell

nvcc -arch=sm_75 compare_memory.cu -o compare_memory
./compare_memory

Comparing Unified Memory vs Explicit Memory Management
Array size: 1048576 elements

Unified Memory - Time: 0.106914s, Max error: 0
Explicit Memory - Time: 0.00928286s, Max error: 0


## Summary of Exercise Answers

### Exercise 2: Thread Execution Order
**Question:** Do `printf()` outputs print in sequential order?

**Answer:** No! The threads execute in parallel and asynchronously. The order depends on:
- Which SM executes which block
- Hardware scheduling decisions
- Warp execution order
- No guaranteed synchronization between threads

This is a fundamental aspect of parallel programming - you cannot assume any execution order unless you explicitly synchronize.

---

### Exercise 3: Multi-Dimensional Indexing
**Question:** Why do `threadIdx.y/z`, `blockIdx.y/z`, etc. exist?

**Answer:** 
- **Purpose**: Map naturally to 2D/3D problems (images, matrices, volumes)
- **Clarity**: Makes code more readable for structured data
- **Optimization**: Helps with memory access patterns

**How to use:**
```cpp
dim3 blockSize(16, 16, 1);  // 256 threads
dim3 gridSize(32, 32, 1);   // 1024 blocks
kernel<<<gridSize, blockSize>>>(...);
```

**Defaults**: Unspecified dimensions default to 1 (for dims) or 0 (for indices)

---

### Exercise 4: GPU Architecture Performance
**Question:** Is Pascal better than K80 for Unified Memory?

**Answer:** Yes! Pascal (Compute 6.0+) introduced the **Page Migration Engine**:
- Automatic page migration between CPU/GPU
- On-demand paging (oversubscription support)
- Hardware page fault handling
- Better concurrent access support

K80 (Kepler) has basic Unified Memory but lacks these hardware features, making it slower and less flexible.

Modern GPUs (Volta, Turing, Ampere, Ada, Hopper) continue to improve on Pascal's foundation.

## Where to From Here

If you enjoyed this notebook and want to learn more, the [NVIDIA DLI](https://nvidia.com/dli) offers several in depth CUDA Programming courses.

For those of you just starting out, please consider [_Fundamentals of Accelerated Computing with CUDA C/C++_](https://courses.nvidia.com/courses/course-v1:DLI+C-AC-01+V1/about) which provides dedicated GPU resources, a more sophisticated programming environment, use of the [NVIDIA Nsight Systems™](https://developer.nvidia.com/nsight-systems) visual profiler, dozens of interactive exercises, detailed presentations, over 8 hours of material, and the ability to earn a DLI Certificate of Competency.

Similarly, for Python programmers, please consider [_Fundamentals of Accelerated Computing with CUDA Python_](https://courses.nvidia.com/courses/course-v1:DLI+C-AC-02+V1/about).

For more intermediate and advance CUDA programming materials, please check out the _Accelerated Computing_ section of the NVIDIA DLI [self-paced catalog](https://www.nvidia.com/en-us/training/online/).